In [2]:
import numpy as np
import numba
import itertools

import sys

# Speed/Memory Tradeoff for using Python ints vs. Numpy.uint8

In [56]:
%timeit 5

6.36 ns ± 0.086 ns per loop (mean ± std. dev. of 7 runs, 100000000 loops each)


In [57]:
sys.getsizeof(5)

28

In [58]:
%timeit np.uint8(5)

260 ns ± 5.7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [59]:
sys.getsizeof(np.uint8(5))

25

All those bytes are going to add up, but no one will ever notice the time.

In [60]:
%timeit 2 + 2

9.33 ns ± 0.231 ns per loop (mean ± std. dev. of 7 runs, 100000000 loops each)


In [61]:
%timeit 2 + np.uint(2)

3.28 µs ± 81.2 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [78]:
%timeit np.uint8(2) + np.uint8(2)

565 ns ± 11.1 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


Pretty sure addition will never be done but clearly arithmetic operations are better with with `np.uint8`'s

# Initial card array building

In [69]:
%timeit np.array([0,0,0,0,0], dtype=np.uint8)

1.78 µs ± 22.9 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [71]:
%timeit np.array([0,0,0,0,0], dtype=int)

1.66 µs ± 22.2 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [34]:
%timeit np.zeros(5, dtype=np.uint8)

1.08 µs ± 5.48 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [45]:
%timeit np.zeros(np.uint(5), dtype=np.uint8)

1.6 µs ± 6.5 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [63]:
%timeit np.zeros(np.uint(5), dtype=int)

1.51 µs ± 9.59 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [64]:
%timeit np.zeros(5, dtype=int)

999 ns ± 7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


There is no point in using `np.uint8`'s as arguments.

In [76]:
sys.getsizeof(np.zeros(5, dtype=np.uint8))

101

In [77]:
sys.getsizeof(np.zeros(5, dtype=int))

136

I'd say the size difference of the different int types is worth the ~600 nanosecond difference in building time.

# Hashing

In [137]:
np.random.seed(12341234)
arr = np.random.randint(low=1, high=53, size=5)

In [138]:
%timeit hash(str(arr))

43.9 µs ± 367 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [139]:
%timeit hash(arr.tostring())

170 ns ± 1.48 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [140]:
%timeit hash(str(arr.data))

926 ns ± 6.92 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [141]:
%timeit hash(str(arr.data.tobytes))

1.21 µs ± 16.1 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


`.tostring()` is the winner.

In [142]:
1 < 2 <3

True

In [143]:
a = np.uint8(5)

In [144]:
b = 5

In [145]:
%timeit a == a

59.1 ns ± 0.496 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [146]:
%timeit b == b

48.5 ns ± 0.461 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [163]:
a = np.arange(start=0, stop=7, dtype=np.uint8)

In [164]:
b = {"not a valid combo","single","double","triple","fullhouse","straight","bomb"}

In [166]:
dict(zip(a,b))

{0: 'single',
 1: 'triple',
 2: 'not a valid combo',
 3: 'fullhouse',
 4: 'double',
 5: 'bomb',
 6: 'straight'}

In [3]:
a = np.arange(start=1, stop=53, dtype=np.uint8)

In [19]:
%timeit b = itertools.combinations(a, 5)

1.27 µs ± 17.8 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [21]:
b = itertools.combinations(a, 5)

In [20]:
combo_dict = dict()

In [22]:
for hand in b:

SyntaxError: unexpected EOF while parsing (<ipython-input-22-db417245d1f3>, line 1)

In [13]:
c[len(c)-1]

(48, 49, 50, 51, 52)

In [27]:
np.uint8(255)

255